In [ ]:
import pandas as pd
import numpy as np
import random
from torch import nn
import torch
import transformers

In [ ]:
!pip install transformers
!git clone https://github.com/hrobjarturh/nlptasks.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'nlptasks'...
remote: Enumerating objects: 384, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 384 (delta 15), reused 36 (delta 12), pack-reused 335
Receiving objects: 100% (384/384), 357.39 MiB | 10.15 MiB/s, done.
Resolving deltas: 100% (201/201), done.
Checking out files: 100% (56/56), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/nlptasks/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/nlptasks/'
/content


In [ ]:
jap_model = torch.load('drive/MyDrive/jap_model.pth',map_location ='cpu')
fin_model = torch.load('drive/MyDrive/fin_model.pth',map_location ='cpu')
eng_model = torch.load('drive/MyDrive/eng_model.pth',map_location ='cpu')

In [ ]:
from transformers import TrainingArguments, Trainer, T5Tokenizer, GPT2Tokenizer, AutoTokenizer
%pip install SentencePiece 

#import tokenizers
eng_tokenizer = AutoTokenizer.from_pretrained('distilgpt2')

jap_tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
jap_tokenizer.do_lower_case = True

fin_tokenizer = AutoTokenizer.from_pretrained('Finnish-NLP/gpt2-finnish'
)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#generate text 
def get_sample(input, model, tokenizer):
  input_ids = tokenizer.encode(input, return_tensors='pt')

  greedy_output = model.generate(input_ids, max_length=50, do_sample=True, top_k=50)
  return (tokenizer.decode(greedy_output[0], skip_special_tokens=True, do_sample=True))



In [ ]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
# import training data
dfte = pd.read_csv('nlptasks/data/dft_eng.csv')
dftj = pd.read_csv('nlptasks/data/dft_jap.csv')
dftf = pd.read_csv('nlptasks/data/dft_fin.csv')


# import validation data
dfve = pd.read_csv('nlptasks/data/dfv_eng.csv')
dfvj = pd.read_csv('nlptasks/data/dfv_jap.csv')
dfvf = pd.read_csv('nlptasks/data/dfv_fin.csv')

In [ ]:
eng_samples = [get_sample((x), eng_model, eng_tokenizer) for x in dfte.question_text[0:10]]
fin_samples = [get_sample((x), fin_model, fin_tokenizer) for x in dftf.question_text[0:10]]
jap_samples = [get_sample((x), jap_model, jap_tokenizer) for x in dftj.question_text[0:10]]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [ ]:
eng_samples

['When was quantum field theory developed?\nQuantum gravity was first described when the theory was first applied for general relativity in 1968, when physicist John von Meier published the first "Quantum Lagrangian" theory (also known as the "G',
 'Who was the first Nobel prize winner for Literature?\nOn October 22 1990, Nobel Prize Winning Professor John Travolta and fellow Nobel Prize-nominee Rene Van der Kees stated that it was possible that Nobel Prize winning authors such',
 'When is the dialectical method used?\nIn the modern era, dialectics has become extremely widely used by practitioners of this discipline, including the Anglican and Anglican Dialectic School. In a 2009 study, one in eight dialectic students',
 'Who invented Hangul?\nShangul has been identified in three books: the book Hui Shāyū (The Man and Woman), "Gangul: A History and History of Gansu",[2] and the first',
 'What do Grasshoppers eat?\nAs in the European Red Cross,[2] the organization serves a diverse range

In [ ]:
jap_samples

['“ダン” ダニエル・ジャドソン・キャラハンの出身はどこ 日本で流通している「赤福」はどこから発祥した? 赤福 - 餅米・白飯・もこ皮が使われ、餡の色',
 'サー・マイケル・フィリップ・ジャガーの出身は? ウィリアム・スミス・クラークは19世紀初頭に何で生まれましたか? トマス・ジェファーソンは1831年12月24日にアラバマ州モービルで生まれた。アラバマ州モービル高校に1',
 'ラオス王国の王族の名前はなんですか? イスラエル・パレスチナ両政府の国境検問所をどこに設けるべきですか? イスラエルと、パレスチナ双方が監視し、双方からそれぞれ1名の検問官が派遣された。パレスチナ側は、',
 '日本テレビ系列『zip!』の初代の司会は誰ですか? フランスの政党には何がありますか? なお、フランスは、フランス共産党 (ps) を中心とする野党が圧倒的な勢力を有しており、野党第3党と議会(下院)の全',
 '西村眞悟弁護士法違反事件で西村が受けた罰則は何? 世界初の人工内耳の開発に成功するのはいつ 1930年代の始め、アメリカのオハイオ州コロンバスにあるシンシナティ・ヒルマン・クリニックでアニー・オコナーとテッド・オコ',
 'ミンデン=リュベッケ郡で最古の風車は何? カール・フリードリヒ・ヴィルヘルム2世はいつ生まれた? 2人の兄弟(カールとフリードリヒ)は7歳で夭折した。 フリードリヒ2世の兄(カールとフリードリヒ)',
 '森上亜希子の出身はどこ 漫画『ドラえもん』はいつ頃から始まった? ドラえもん のび太のしっぽ ドラえもん のび太の人食いニンジン ドラえもん のび太と緑の巨人伝 ドラえもん のび太 のび太の',
 '相模原線が開通したのはいつ ニュージャパンプロレスはいつ誕生した? 新日本で初代王者になったのは橋本真也。',
 'ロジャー・フェデラーはいつ生まれた リチャード・ブラッドリー・ラッセルの出身はどこ リチャード・ブラッドリー・ラッセル(richard bradley russell, 1766年5月14日 - 1818年4',
 'ジョージ・ローレンス・マイカンはいつ生まれた? フランスの天文学者は誰ですか? ピエール・ベルジェは1786年にフランスのパリを出発した[1]。この時点において、彼はフランス天文学の発展のほとんどを主導していたばかり']

In [ ]:
fin_samples

['Milloin Charles Fort syntyi?\nLucy Marie Franklin, 1766\n\n� (2. joulukuuta 1807 Lyon, Ranska) olisi yhdysvaltalainen matemaatikko ja matemaatikko. Hänen nimeään ei myöskään tunneta tarkasti. Hän opiskeli matemati',
 'Mitä on altruismi?\n\nAmeriikan tutkimuksen kannalta altruismi on luonteeltaan luonteeltaan poliittinen käsite, joka viittaa maailmankatsomukselliseen suuntaukseen eli näkemykseen, jossa ei ole mitään uskonnollista tai ideologista, mutta joka kuitenkin mahdollistaa uskon',
 'Mikä oli Wilhelm Wagner viimeinen sävellys?\nLike: The Hymnstone of the Hymnstone of the Hymnstone of the Hymnstone of the Hymnstone of the Hymnstone of',
 'Missä Harz sijaitsee?\nKeväällä 2015 valmistui Harzin viides tuotantokausi. Sarja jatkuu neljällä kaudella, johon sisältyi kaksi neljäsosaa. Sarjan kuudennen kaudella on jälleen vuorossa uusi jakso nimeltään Our year and year. Tällä kertaa vuorossa olevat kausi toisensa',
 'Miten paljon täysikasvuinen ahma painaa?\n]�\nSuuri ahma on 6,6 kilogram